# Tratamento e Manipulação dos Dados de Transações PicPay

Esse case técnico têm por objetivo garantir a integridade dos dados necessários para os cálculos da receita obtida em diferentes tipos de transações realizadas pelo PicPay, além de gerar arquivos em CSV com os resultados alcançados.

**As tarefas realizadas foram divididas em:**
1. Importação e inspeção dos dados
2. Cálculos do dataset "transactions"
3. Criação do dataset "transactions_installments"
4. Criação e exportação dos arquivos finais em CSV

**Requisitos:**
- Python 3.12+: Linguagem usada para análise e manipulação de dados.
- Pandas: Manipulação de tabelas e dados estruturados.
- NumPy: Operações matemáticas e com arrays.
- os: Acesso a funcionalidades do sistema operacional.

## 1. Importação e inspeção dos dados

### Importação de Bibliotecas

In [163]:
# Bibliotecas
import pandas as pd
import numpy as np

# Configurações 
pd.set_option('display.max_columns', None)

### Importação dos Dados

In [164]:
transactions = pd.read_csv('transactions.csv', sep=';')

In [165]:
transactions.head(10)

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate
0,1,03/01/2021,P2P,400,600,Credit card,12,1.99,2.99,3.49
1,2,14/09/2021,BILLS,650,300,Credit card,5,1.99,2.99,3.49
2,3,20/07/2021,BILLS,1200,0,Credit card,8,1.99,2.99,3.49
3,4,06/08/2021,P2P,350,800,Credit card,9,1.99,2.99,3.49
4,5,13/04/2021,P2P,3500,0,Credit card,10,1.99,2.99,3.49
5,6,24/05/2021,P2P,3420,0,Credit card,7,1.99,2.99,3.49
6,7,31/03/2021,BILLS,5000,0,Credit card,12,1.99,2.99,3.49
7,8,30/08/2021,P2P,2800,0,Credit card,12,1.99,2.99,3.49
8,9,28/08/2021,BILLS,6000,0,Credit card,4,1.99,2.99,3.49
9,10,27/09/2021,P2P,8000,0,Credit card,6,1.99,2.99,3.49


### Inspeção dos Dados

Buscou-se checar a integridade do dataset através dos os tipos de cada coluna, valores nulos e valores duplicados.

In [166]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          10 non-null     int64  
 1   transaction_date        10 non-null     object 
 2   transaction_type        10 non-null     object 
 3   transaction_value       10 non-null     int64  
 4   receiver_used_cc_limit  10 non-null     int64  
 5   payment_method          10 non-null     object 
 6   installments            10 non-null     int64  
 7   p2p_surcharge_rate      10 non-null     float64
 8   bills_surcharge_rate    10 non-null     float64
 9   installment_rate        10 non-null     float64
dtypes: float64(3), int64(4), object(3)
memory usage: 932.0+ bytes


In [167]:
transactions.isna().sum()

transaction_id            0
transaction_date          0
transaction_type          0
transaction_value         0
receiver_used_cc_limit    0
payment_method            0
installments              0
p2p_surcharge_rate        0
bills_surcharge_rate      0
installment_rate          0
dtype: int64

In [168]:
transactions.duplicated().sum()

0

Notou-se que a coluna "transaction_date" foi erroneamente lida como objeto, foi feita a conversão para o tipo correto de data em pandas (datetime64).

In [169]:
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'], dayfirst=True)

In [170]:
transactions.head()

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate
0,1,2021-01-03,P2P,400,600,Credit card,12,1.99,2.99,3.49
1,2,2021-09-14,BILLS,650,300,Credit card,5,1.99,2.99,3.49
2,3,2021-07-20,BILLS,1200,0,Credit card,8,1.99,2.99,3.49
3,4,2021-08-06,P2P,350,800,Credit card,9,1.99,2.99,3.49
4,5,2021-04-13,P2P,3500,0,Credit card,10,1.99,2.99,3.49


In [171]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   transaction_id          10 non-null     int64         
 1   transaction_date        10 non-null     datetime64[ns]
 2   transaction_type        10 non-null     object        
 3   transaction_value       10 non-null     int64         
 4   receiver_used_cc_limit  10 non-null     int64         
 5   payment_method          10 non-null     object        
 6   installments            10 non-null     int64         
 7   p2p_surcharge_rate      10 non-null     float64       
 8   bills_surcharge_rate    10 non-null     float64       
 9   installment_rate        10 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(4), object(2)
memory usage: 932.0+ bytes


Os dados estão em seus tipos corretos e não possuem registros nulos ou duplicados, podemos seguir para etapas de cálculos.

## 2. Cálculos do dataset "transactions"

Foi criado um fluxograma para melhor compreensão da jornada que uma transação percorre para definição final da receita gerada, usando as definições das regras de negócio oferecidas para o case técnico e extrapolações para cenários não representados nos dados oferecidos para teste.

<p align="center">
  <img src="imgs//fluxo_transacoes.png" alt="Imagem também disponível na pasta imgs desse projeto" width="900">
</p>

\* Como a metodologia de cálculo de juros de parcelamento não foi explicitada para P2P, considerou-se a mesma de BILLS

\*\* Considerou-se que pagamentos BILLS usando ambos saldo em carteira e cartão de crédito sejam representados por dois registros de mesmo ID e método de pagamento distinto, de forma que os cálculos para cada parte seriam realizados separadamente conforme as regras de negócio

In [172]:
# Cálculo de valor das taxas adicionais para transações P2P (p2p_surcharge) e BILLS (bills_surcharge)

# Valor da taxa P2P
transactions['p2p_surcharge'] = np.where(
    (   
        # Filtrando transações do tipo P2P, com pagamento por cartão de crédito e com extrapolação do limite
        (transactions['transaction_type'] == 'P2P') & 
        (transactions['payment_method'] == 'Credit card') &
        (transactions['transaction_value'] + transactions['receiver_used_cc_limit'] > 800)
    ),
    # Aplicação da taxa no valor excedente
    (transactions['transaction_value'] + transactions['receiver_used_cc_limit'] - 800) * transactions['p2p_surcharge_rate']/100,
    0
)

# Valor da taxa BILLS
transactions['bills_surcharge'] = np.where(
    (   
        # Filtrando transações do tipo BILLS e com pagamento por cartão de crédito 
        (transactions['transaction_type'] == 'BILLS') &
        (transactions['payment_method'] == 'Credit card')
    ),
    # Aplicação da taxa no valor total
    (transactions['transaction_value'] * transactions['bills_surcharge_rate'])/100,
    0
)

# Arredondando valores para casas decimais comumente usadas nesse contexto
transactions['p2p_surcharge'] = np.round(transactions['p2p_surcharge'], 2)
transactions['bills_surcharge'] = np.round(transactions['bills_surcharge'], 2)

transactions.head(10)

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate,p2p_surcharge,bills_surcharge
0,1,2021-01-03,P2P,400,600,Credit card,12,1.99,2.99,3.49,3.98,0.00
1,2,2021-09-14,BILLS,650,300,Credit card,5,1.99,2.99,3.49,0.00,19.44
2,3,2021-07-20,BILLS,1200,0,Credit card,8,1.99,2.99,3.49,0.00,35.88
3,4,2021-08-06,P2P,350,800,Credit card,9,1.99,2.99,3.49,6.96,0.00
4,5,2021-04-13,P2P,3500,0,Credit card,10,1.99,2.99,3.49,53.73,0.00
5,6,2021-05-24,P2P,3420,0,Credit card,7,1.99,2.99,3.49,52.14,0.00
6,7,2021-03-31,BILLS,5000,0,Credit card,12,1.99,2.99,3.49,0.00,149.50
7,8,2021-08-30,P2P,2800,0,Credit card,12,1.99,2.99,3.49,39.80,0.00
8,9,2021-08-28,BILLS,6000,0,Credit card,4,1.99,2.99,3.49,0.00,179.40
9,10,2021-09-27,P2P,8000,0,Credit card,6,1.99,2.99,3.49,143.28,0.00


In [173]:
# Cálculo de valor total da transação incluíndo taxas P2P e BILLS (surcharged_transaction_value)
transactions['surcharged_transaction_value'] = transactions['transaction_value'] + transactions['p2p_surcharge'] + transactions['bills_surcharge']  

transactions.head(10)

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate,p2p_surcharge,bills_surcharge,surcharged_transaction_value
0,1,2021-01-03,P2P,400,600,Credit card,12,1.99,2.99,3.49,3.98,0.00,403.98
1,2,2021-09-14,BILLS,650,300,Credit card,5,1.99,2.99,3.49,0.00,19.44,669.44
2,3,2021-07-20,BILLS,1200,0,Credit card,8,1.99,2.99,3.49,0.00,35.88,1235.88
3,4,2021-08-06,P2P,350,800,Credit card,9,1.99,2.99,3.49,6.96,0.00,356.96
4,5,2021-04-13,P2P,3500,0,Credit card,10,1.99,2.99,3.49,53.73,0.00,3553.73
5,6,2021-05-24,P2P,3420,0,Credit card,7,1.99,2.99,3.49,52.14,0.00,3472.14
6,7,2021-03-31,BILLS,5000,0,Credit card,12,1.99,2.99,3.49,0.00,149.50,5149.50
7,8,2021-08-30,P2P,2800,0,Credit card,12,1.99,2.99,3.49,39.80,0.00,2839.80
8,9,2021-08-28,BILLS,6000,0,Credit card,4,1.99,2.99,3.49,0.00,179.40,6179.40
9,10,2021-09-27,P2P,8000,0,Credit card,6,1.99,2.99,3.49,143.28,0.00,8143.28


In [174]:
# Cálculo do valor de cada parcela a ser paga pelo usuário que fez um pagamento parcelado com cartão de crédito (individual_installment)

s0 = transactions['surcharged_transaction_value'] # saldo devedor no momento da transação
j = transactions['installment_rate']/100          # taxa de juros     
n = transactions['installments']                  # número de parcelas

transactions['individual_installment'] = np.where(
    (
        # Filtrando transações por cartão de crédito com mais de uma parcela
        (transactions['payment_method'] == 'Credit card') &
        (transactions['installments'] > 1) 
    ),
    # Aplicando fórmula de juros compostos para definir a parcela fixa
    (s0 * (j/(1 - (1 + j)**(-n)))),
    0
)

# Arredondando valores
transactions['individual_installment'] = np.round(transactions['individual_installment'], 2)

transactions.head(10)

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate,p2p_surcharge,bills_surcharge,surcharged_transaction_value,individual_installment
0,1,2021-01-03,P2P,400,600,Credit card,12,1.99,2.99,3.49,3.98,0.00,403.98,41.78
1,2,2021-09-14,BILLS,650,300,Credit card,5,1.99,2.99,3.49,0.00,19.44,669.44,148.23
2,3,2021-07-20,BILLS,1200,0,Credit card,8,1.99,2.99,3.49,0.00,35.88,1235.88,179.72
3,4,2021-08-06,P2P,350,800,Credit card,9,1.99,2.99,3.49,6.96,0.00,356.96,46.90
4,5,2021-04-13,P2P,3500,0,Credit card,10,1.99,2.99,3.49,53.73,0.00,3553.73,427.09
5,6,2021-05-24,P2P,3420,0,Credit card,7,1.99,2.99,3.49,52.14,0.00,3472.14,567.64
6,7,2021-03-31,BILLS,5000,0,Credit card,12,1.99,2.99,3.49,0.00,149.50,5149.50,532.58
7,8,2021-08-30,P2P,2800,0,Credit card,12,1.99,2.99,3.49,39.80,0.00,2839.80,293.70
8,9,2021-08-28,BILLS,6000,0,Credit card,4,1.99,2.99,3.49,0.00,179.40,6179.40,1681.95
9,10,2021-09-27,P2P,8000,0,Credit card,6,1.99,2.99,3.49,143.28,0.00,8143.28,1527.73


In [175]:
# Cálculo do valor total do juros a ser cobrado pelo parcelamento

transactions['installment_fee'] = np.where(
    (
        # Filtrando transações por cartão de crédito com mais de uma parcela
        (transactions['payment_method'] == 'Credit card') &
        (transactions['installments'] > 1) 
    ),
    # Calculando diferença entre valor total da transação depois e antes da aplicação dos juros
    transactions['individual_installment'] * transactions['installments'] - transactions['surcharged_transaction_value'],
    0
)

transactions.head(10)

,transaction_id,transaction_date,transaction_type,transaction_value,receiver_used_cc_limit,payment_method,installments,p2p_surcharge_rate,bills_surcharge_rate,installment_rate,p2p_surcharge,bills_surcharge,surcharged_transaction_value,individual_installment,installment_fee
0,1,2021-01-03,P2P,400,600,Credit card,12,1.99,2.99,3.49,3.98,0.00,403.98,41.78,97.38
1,2,2021-09-14,BILLS,650,300,Credit card,5,1.99,2.99,3.49,0.00,19.44,669.44,148.23,71.71
2,3,2021-07-20,BILLS,1200,0,Credit card,8,1.99,2.99,3.49,0.00,35.88,1235.88,179.72,201.88
3,4,2021-08-06,P2P,350,800,Credit card,9,1.99,2.99,3.49,6.96,0.00,356.96,46.90,65.14
4,5,2021-04-13,P2P,3500,0,Credit card,10,1.99,2.99,3.49,53.73,0.00,3553.73,427.09,717.17
5,6,2021-05-24,P2P,3420,0,Credit card,7,1.99,2.99,3.49,52.14,0.00,3472.14,567.64,501.34
6,7,2021-03-31,BILLS,5000,0,Credit card,12,1.99,2.99,3.49,0.00,149.50,5149.50,532.58,1241.46
7,8,2021-08-30,P2P,2800,0,Credit card,12,1.99,2.99,3.49,39.80,0.00,2839.80,293.70,684.60
8,9,2021-08-28,BILLS,6000,0,Credit card,4,1.99,2.99,3.49,0.00,179.40,6179.40,1681.95,548.40
9,10,2021-09-27,P2P,8000,0,Credit card,6,1.99,2.99,3.49,143.28,0.00,8143.28,1527.73,1023.10


## 3. Criação do dataset "transactions_installments"

O objetivo desse dataset é a divisão das transações parceladas em suas parcelas individuais, além do cálculo dos valores definidos pela tabela PRICE (amortização e juros).

In [176]:
# Criando novo DataFrame

# Definindo colunas a serem removidas 
drop_columns = [
        'p2p_surcharge_rate', 
        'bills_surcharge_rate', 
        'installment_rate', 
        'p2p_surcharge', 
        'bills_surcharge', 
        'surcharged_transaction_value',
        'installments',
        'installment_fee'
        ]

# Definindo colunas a serem renomeadas 
rename_columns = {
    'receiver_used_cc_limit': 'used_cc_limit',
    'individual_installment': 'due_amount'
}

# Cria novo dataframe contendo apenas "transactions" parceladas
transactions_installments = transactions[transactions['installments'] > 1].copy()
# Cria coluna com listas com valores inteiros de 1 ao número total de parcelas
transactions_installments['installment_number'] = transactions_installments['installments'].apply(lambda x: list(range(1, int(x) + 1)))
# "Explode" a coluna de listas, transformando cada parcela em uma nova linha
transactions_installments = transactions_installments.explode('installment_number')
# Converte os valores da coluna 'installment_number' para o tipo inteiro
transactions_installments['installment_number'] = transactions_installments['installment_number'].astype(int)
# Reseta o índice do DataFrame 
transactions_installments = transactions_installments.reset_index(drop=True)
# Remove colunas desnecessárias
transactions_installments = transactions_installments.drop(drop_columns, axis='columns')
# Renomeia colunas
transactions_installments = transactions_installments.rename(columns=rename_columns)

transactions_installments.head(12)

,transaction_id,transaction_date,transaction_type,transaction_value,used_cc_limit,payment_method,due_amount,installment_number
0,1,2021-01-03,P2P,400,600,Credit card,41.78,1
1,1,2021-01-03,P2P,400,600,Credit card,41.78,2
2,1,2021-01-03,P2P,400,600,Credit card,41.78,3
3,1,2021-01-03,P2P,400,600,Credit card,41.78,4
4,1,2021-01-03,P2P,400,600,Credit card,41.78,5
5,1,2021-01-03,P2P,400,600,Credit card,41.78,6
6,1,2021-01-03,P2P,400,600,Credit card,41.78,7
7,1,2021-01-03,P2P,400,600,Credit card,41.78,8
8,1,2021-01-03,P2P,400,600,Credit card,41.78,9
9,1,2021-01-03,P2P,400,600,Credit card,41.78,10


Neste projeto, considerou-se que a primeira parcela é cobrada na data da transação, enquanto as parcelas seguintes são cobradas mensalmente na mesma data.

In [177]:
# Cálculo da data de vencimento da prestação

transactions_installments['due_date'] = transactions_installments.apply(
    lambda row: row['transaction_date'] + pd.DateOffset(months=row['installment_number'] - 1),
    axis=1
)

transactions_installments.head(12)

,transaction_id,transaction_date,transaction_type,transaction_value,used_cc_limit,payment_method,due_amount,installment_number,due_date
0,1,2021-01-03,P2P,400,600,Credit card,41.78,1,2021-01-03
1,1,2021-01-03,P2P,400,600,Credit card,41.78,2,2021-02-03
2,1,2021-01-03,P2P,400,600,Credit card,41.78,3,2021-03-03
3,1,2021-01-03,P2P,400,600,Credit card,41.78,4,2021-04-03
4,1,2021-01-03,P2P,400,600,Credit card,41.78,5,2021-05-03
5,1,2021-01-03,P2P,400,600,Credit card,41.78,6,2021-06-03
6,1,2021-01-03,P2P,400,600,Credit card,41.78,7,2021-07-03
7,1,2021-01-03,P2P,400,600,Credit card,41.78,8,2021-08-03
8,1,2021-01-03,P2P,400,600,Credit card,41.78,9,2021-09-03
9,1,2021-01-03,P2P,400,600,Credit card,41.78,10,2021-10-03


Para definir os valores de amortização e juros por parcela, primeiro vamos definir uma função para aplicar os cálculos da tabela PRICE.

In [178]:
def calculate_installments(df):
    """
    Esta função calcula o saldo devedor, a amortização da parcela, a taxa de juros da parcela
    e o valor devido para cada parcela de uma transação, utilizando a fórmula da Tabela Price.

    Parâmetros:
    - df: Um dataframe de parcelas para uma transação específica (agrupado por 'transaction_id').

    Retorna:
    - calculated_values: Um DataFrame contendo os valores calculados para cada parcela.
    """
    balance = df['surcharged_transaction_value'].iloc[0]      # Saldo devedor inicial
    interest_rate = df['installment_rate'].iloc[0] / 100      # Taxa de juros
    installment_value = df['individual_installment'].iloc[0]  # Valor fixo da prestação
    num_installments = df['installments'].iloc[0]             # Número total de parcelas

    calculated_values = []

    
    # Definindo valores calculados para cada parcela 
    for i in range(num_installments):
        interest = balance * interest_rate # juro
        amortization = installment_value - interest # amortização

        # Popula lista com valores calculados
        calculated_values.append({
            'loan_balance': balance,
            'installment_fee': interest,
            'installment_amortization': amortization,
            'due_amount': installment_value
        })
        balance -= amortization  # reduz a amortização do valor de saldo devedor para próxima parcela

    return pd.DataFrame(calculated_values) # converte para pandas DataFrame

Para obtermos os valores de saldo devedor inicial, taxa de juros, número de parcelas e valor fixo da prestação, necessários para os cálculos do sistema de amortização Francês, realizou-se um merge do dataframe "transactions_installments" com "transactions" através do ID da transação.

In [179]:
# Merge de DataFrames com colunas de interesse através do transaction_id
transactions_installments = transactions_installments.merge(
    transactions[['transaction_id', 'surcharged_transaction_value', 'installment_rate', 'installments', 'individual_installment']],
    on='transaction_id',
    how='left'
)

# Criação das colunas de saldo devedor, amortização, juros e prestação 
transactions_installments['loan_balance'] = np.nan
transactions_installments['installment_amortization'] = np.nan
transactions_installments['installment_fee'] = np.nan
transactions_installments['due_amount'] = np.nan

# Aplica a função para cada transaction_id
calculated_values = transactions_installments.groupby('transaction_id', group_keys=False).apply(calculate_installments).reset_index(drop=True)

# Popula colunas com valores calculados correspondentes
transactions_installments[['loan_balance', 'installment_fee', 'installment_amortization', 'due_amount']] = calculated_values

# Colunas para arredondar
round_columns = [
    'loan_balance', 
    'installment_fee', 
    'installment_amortization', 
    'due_amount'
]

# Colunas para remover
drop_columns = [
    'surcharged_transaction_value',
    'installment_rate',
    'installments'
]

# Arredonda valores de colunas definidas
transactions_installments[round_columns] = \
    transactions_installments[round_columns].apply(lambda x: np.round(x, 2))

# Remove colunas definidas
transactions_installments = transactions_installments.drop(drop_columns, axis='columns')

transactions_installments.head(12)

C:\Users\taina\AppData\Local\Temp\ipykernel_2304\4019344977.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  calculated_values = transactions_installments.groupby('transaction_id', group_keys=False).apply(calculate_installments).reset_index(drop=True)


,transaction_id,transaction_date,transaction_type,transaction_value,used_cc_limit,payment_method,due_amount,installment_number,due_date,individual_installment,loan_balance,installment_amortization,installment_fee
0,1,2021-01-03,P2P,400,600,Credit card,41.78,1,2021-01-03,41.78,403.98,27.68,14.10
1,1,2021-01-03,P2P,400,600,Credit card,41.78,2,2021-02-03,41.78,376.30,28.65,13.13
2,1,2021-01-03,P2P,400,600,Credit card,41.78,3,2021-03-03,41.78,347.65,29.65,12.13
3,1,2021-01-03,P2P,400,600,Credit card,41.78,4,2021-04-03,41.78,318.00,30.68,11.10
4,1,2021-01-03,P2P,400,600,Credit card,41.78,5,2021-05-03,41.78,287.32,31.75,10.03
5,1,2021-01-03,P2P,400,600,Credit card,41.78,6,2021-06-03,41.78,255.57,32.86,8.92
6,1,2021-01-03,P2P,400,600,Credit card,41.78,7,2021-07-03,41.78,222.71,34.01,7.77
7,1,2021-01-03,P2P,400,600,Credit card,41.78,8,2021-08-03,41.78,188.70,35.19,6.59
8,1,2021-01-03,P2P,400,600,Credit card,41.78,9,2021-09-03,41.78,153.51,36.42,5.36
9,1,2021-01-03,P2P,400,600,Credit card,41.78,10,2021-10-03,41.78,117.09,37.69,4.09


## 4. Criação e exportação dos arquivos finais

### Inspeção final dos DataFrames

Para garantir a integridade dos datasets gerados neste notebook, checamos informações de contagem de não nulos e tipos de dados de cada coluna, para cada um deles. 

In [180]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   transaction_id                10 non-null     int64         
 1   transaction_date              10 non-null     datetime64[ns]
 2   transaction_type              10 non-null     object        
 3   transaction_value             10 non-null     int64         
 4   receiver_used_cc_limit        10 non-null     int64         
 5   payment_method                10 non-null     object        
 6   installments                  10 non-null     int64         
 7   p2p_surcharge_rate            10 non-null     float64       
 8   bills_surcharge_rate          10 non-null     float64       
 9   installment_rate              10 non-null     float64       
 10  p2p_surcharge                 10 non-null     float64       
 11  bills_surcharge               10 no

In [181]:
transactions_installments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   transaction_id            85 non-null     int64         
 1   transaction_date          85 non-null     datetime64[ns]
 2   transaction_type          85 non-null     object        
 3   transaction_value         85 non-null     int64         
 4   used_cc_limit             85 non-null     int64         
 5   payment_method            85 non-null     object        
 6   due_amount                85 non-null     float64       
 7   installment_number        85 non-null     int32         
 8   due_date                  85 non-null     datetime64[ns]
 9   individual_installment    85 non-null     float64       
 10  loan_balance              85 non-null     float64       
 11  installment_amortization  85 non-null     float64       
 12  installment_fee         

### Exportação dos Dataframes finais em CSV

Por fim, exportamos os datasets finais em formato CSV com destino ao mesmo diretório deste projeto. 

In [182]:
import os
transactions_complete = transactions.copy() # Mudança de nome para garantir que o dataset original não seja sobrescrito

# Canvertendo dataframes para CSV e salvando no diretório do projeto
transactions_complete.to_csv('transactions_complete.csv', index=False)
transactions_installments.to_csv('transactions_installments.csv', index=False)